Radek posted about this [here](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/discussion/309220), and linked to a GitHub repo with the code.

I just transferred that code here to Kaggle notebooks, that's all.

In [86]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [87]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []
        
    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)


def calculate_apk(list_of_preds, list_of_gts):
    # for fast validation this can be changed to operate on dicts of {'cust_id_int': [art_id_int, ...]}
    # using 'data/val_week_purchases_by_cust.pkl'
    apks = []
    for preds, gt in zip(list_of_preds, list_of_gts):
        apks.append(apk(gt, preds, k=12))
    return np.mean(apks)

def eval_sub(sub_csv, skip_cust_with_no_purchases=True):
    sub=pd.read_csv(sub_csv)
    validation_set=pd.read_parquet('data/validation_ground_truth.parquet')

    apks = []

    no_purchases_pattern = []
    for pred, gt in zip(sub.prediction.str.split(), validation_set.prediction.str.split()):
        if skip_cust_with_no_purchases and (gt == no_purchases_pattern): continue
        apks.append(apk(gt, pred, k=12))
    return np.mean(apks)

In [88]:
import pandas as pd

In [89]:
%%time

transactions = pd.read_parquet('../input/warmup/transactions_train.parquet')
customers = pd.read_parquet('../input/warmup/customers.parquet')
articles = pd.read_parquet('../input/warmup/articles.parquet')

# sample = 0.05
# transactions = pd.read_parquet(f'data/transactions_train_sample_{sample}.parquet')
# customers = pd.read_parquet(f'data/customers_sample_{sample}.parquet')
# articles = pd.read_parquet(f'data/articles_train_sample_{sample}.parquet')

CPU times: user 2.33 s, sys: 4.48 s, total: 6.81 s
Wall time: 8.5 s


In [90]:
test_week = transactions.week.max() + 1
transactions = transactions[transactions.week > transactions.week.max() - 10]

# Generating candidates

### Last purchase candidates

In [91]:
%%time

c2weeks = transactions.groupby('customer_id')['week'].unique()

CPU times: user 18.3 s, sys: 506 ms, total: 18.8 s
Wall time: 18.4 s


In [92]:
transactions.groupby('week')['t_dat'].agg(['min', 'max'])

,min,max
week,,
95,2020-07-15,2020-07-21
96,2020-07-22,2020-07-28
97,2020-07-29,2020-08-04
98,2020-08-05,2020-08-11
99,2020-08-12,2020-08-18
100,2020-08-19,2020-08-25
101,2020-08-26,2020-09-01
102,2020-09-02,2020-09-08
103,2020-09-09,2020-09-15


In [93]:
c2weeks

customer_id
28847241659200          [95, 96, 101, 102]
41318098387474                        [98]
116809474287335                 [101, 103]
200292573348128          [95, 96, 99, 102]
248294615847351                       [96]
                               ...        
18446624797007271432                  [95]
18446630855572834764                 [103]
18446662237889060501                 [100]
18446705133201055310                 [102]
18446737527580148316                 [104]
Name: week, Length: 437365, dtype: object

In [94]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

CPU times: user 1.13 s, sys: 188 ms, total: 1.32 s
Wall time: 1.31 s


In [95]:
c2weeks2shifted_weeks[28847241659200]

{95: 96, 96: 101, 101: 102, 102: 105}

In [96]:
candidates_last_purchase = transactions.copy()

In [97]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])
    
candidates_last_purchase.week=weeks

CPU times: user 7.46 s, sys: 23 ms, total: 7.49 s
Wall time: 7.49 s


In [98]:
candidates_last_purchase[candidates_last_purchase['customer_id']==272412481300040]

,t_dat,customer_id,article_id,price,sales_channel_id,week
29030503,2020-07-15,272412481300040,778064028,0.008458,1,96
29030504,2020-07-15,272412481300040,816592008,0.016932,1,96
29030505,2020-07-15,272412481300040,621381021,0.033881,1,96
29030506,2020-07-15,272412481300040,817477003,0.025407,1,96
29030507,2020-07-15,272412481300040,899088002,0.025407,1,96
29319533,2020-07-22,272412481300040,885077001,0.008458,1,103
29410772,2020-07-24,272412481300040,850176003,0.029034,2,103
29410773,2020-07-24,272412481300040,875803001,0.064559,2,103
29410774,2020-07-24,272412481300040,892970003,0.020966,2,103
29410775,2020-07-24,272412481300040,854619003,0.020966,2,103


In [99]:
transactions[transactions['customer_id']==272412481300040]

,t_dat,customer_id,article_id,price,sales_channel_id,week
29030503,2020-07-15,272412481300040,778064028,0.008458,1,95
29030504,2020-07-15,272412481300040,816592008,0.016932,1,95
29030505,2020-07-15,272412481300040,621381021,0.033881,1,95
29030506,2020-07-15,272412481300040,817477003,0.025407,1,95
29030507,2020-07-15,272412481300040,899088002,0.025407,1,95
29319533,2020-07-22,272412481300040,885077001,0.008458,1,96
29410772,2020-07-24,272412481300040,850176003,0.029034,2,96
29410773,2020-07-24,272412481300040,875803001,0.064559,2,96
29410774,2020-07-24,272412481300040,892970003,0.020966,2,96
29410775,2020-07-24,272412481300040,854619003,0.020966,2,96


### Bestsellers candidates

In [100]:
mean_price = transactions \
    .groupby(['week', 'article_id'])['price'].mean()

In [101]:
mean_price

week  article_id
95    108775015     0.004729
      108775044     0.008458
      110065001     0.006085
      110065002     0.006085
      111565001     0.004288
                      ...   
104   952267001     0.013732
      952938001     0.048651
      953450001     0.016932
      953763001     0.021885
      956217002     0.059068
Name: price, Length: 196880, dtype: float32

In [102]:
sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')

In [103]:
sales

week  article_id
95    760084003      1
      866731001      2
      600886001      3
      706016001      4
      372860002      5
                    ..
104   915529003      8
      915529005      9
      448509014     10
      762846027     11
      714790020     12
Name: bestseller_rank, Length: 120, dtype: int8

In [104]:
sales.loc[95]

article_id
760084003     1
866731001     2
600886001     3
706016001     4
372860002     5
610776002     6
877278002     7
547780003     8
817354001     9
827968001    10
866731003    11
866383006    12
Name: bestseller_rank, dtype: int8

In [105]:
bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1

In [106]:
bestsellers_previous_week.pipe(lambda df: df[df['week']==96])

,week,article_id,bestseller_rank,price
0,96,760084003,1,0.025094
1,96,866731001,2,0.024919
2,96,600886001,3,0.022980
3,96,706016001,4,0.033197
4,96,372860002,5,0.013193
5,96,610776002,6,0.008318
6,96,877278002,7,0.025036
7,96,547780003,8,0.024814
8,96,817354001,9,0.021913
9,96,827968001,10,0.016436


In [107]:
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()

In [108]:
unique_transactions

,t_dat,customer_id,sales_channel_id,week
29030503,2020-07-15,272412481300040,1,95
29064059,2020-07-15,1456826891333599,1,95
29067103,2020-07-15,2133687643102426,2,95
29027487,2020-07-15,6010692573790711,1,95
29046403,2020-07-15,6171059100114610,2,95
...,...,...,...,...
31760188,2020-09-22,18435221511488011015,1,104
31782234,2020-09-22,18436859303155335645,1,104
31787251,2020-09-22,18437941771381362708,2,104
31776022,2020-09-22,18438270306572912089,1,104


In [109]:
transactions.drop_duplicates(['week', 'customer_id'])

,t_dat,customer_id,article_id,price,sales_channel_id,week
29030503,2020-07-15,272412481300040,778064028,0.008458,1,95
29064059,2020-07-15,1456826891333599,888294001,0.013542,1,95
29067103,2020-07-15,2133687643102426,843642001,0.042356,2,95
29027487,2020-07-15,6010692573790711,857812010,0.039661,1,95
29046403,2020-07-15,6171059100114610,815447007,0.006763,2,95
...,...,...,...,...,...,...
31760188,2020-09-22,18435221511488011015,573085055,0.033881,1,104
31782234,2020-09-22,18436859303155335645,801447001,0.030492,1,104
31787251,2020-09-22,18437941771381362708,907188001,0.050831,2,104
31776022,2020-09-22,18438270306572912089,751471043,0.033881,1,104


In [110]:
candidates_bestsellers = pd.merge(
    unique_transactions,
    bestsellers_previous_week,
    on='week',
)

In [111]:
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

In [112]:
test_set_transactions

,t_dat,customer_id,sales_channel_id,week
0,2020-07-15,272412481300040,1,105
1,2020-07-15,1456826891333599,1,105
2,2020-07-15,2133687643102426,2,105
3,2020-07-15,6010692573790711,1,105
4,2020-07-15,6171059100114610,2,105
...,...,...,...,...
437360,2020-09-22,18410229429441241008,2,105
437361,2020-09-22,18417769707947924979,2,105
437362,2020-09-22,18418054986721795659,2,105
437363,2020-09-22,18421175435799911749,2,105


In [113]:
candidates_bestsellers_test_week = pd.merge(
    test_set_transactions,
    bestsellers_previous_week,
    on='week'
)

In [114]:
candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

In [115]:
candidates_bestsellers

,t_dat,customer_id,sales_channel_id,week,article_id,price
0,2020-07-22,200292573348128,2,96,760084003,0.025094
1,2020-07-22,200292573348128,2,96,866731001,0.024919
2,2020-07-22,200292573348128,2,96,600886001,0.022980
3,2020-07-22,200292573348128,2,96,706016001,0.033197
4,2020-07-22,200292573348128,2,96,372860002,0.013193
...,...,...,...,...,...,...
5248375,2020-09-22,18438270306572912089,1,105,915529003,0.033439
5248376,2020-09-22,18438270306572912089,1,105,915529005,0.033417
5248377,2020-09-22,18438270306572912089,1,105,448509014,0.041630
5248378,2020-09-22,18438270306572912089,1,105,762846027,0.025005


# Combining transactions and candidates / negative examples

In [116]:
transactions['purchased'] = 1

In [117]:
data = pd.concat([transactions, candidates_last_purchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)

In [118]:
data

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased
29030503,2020-07-15,272412481300040,778064028,0.008458,1,95,1.0
29030504,2020-07-15,272412481300040,816592008,0.016932,1,95,1.0
29030505,2020-07-15,272412481300040,621381021,0.033881,1,95,1.0
29030506,2020-07-15,272412481300040,817477003,0.025407,1,95,1.0
29030507,2020-07-15,272412481300040,899088002,0.025407,1,95,1.0
...,...,...,...,...,...,...,...
5248375,2020-09-22,18438270306572912089,915529003,0.033439,1,105,0.0
5248376,2020-09-22,18438270306572912089,915529005,0.033417,1,105,0.0
5248377,2020-09-22,18438270306572912089,448509014,0.041630,1,105,0.0
5248378,2020-09-22,18438270306572912089,762846027,0.025005,1,105,0.0


In [119]:
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

In [120]:
data.purchased.mean()

0.13607582749165664

### Engineering and adding new features

#### Customer Purchase Frequency

In [121]:
customer_frequencies = transactions[['customer_id']].copy()
customer_frequencies['customer_frequency'] = customer_frequencies['customer_id'].map(customer_frequencies['customer_id'].value_counts())

In [122]:
customer_frequencies['customer_frequency'].value_counts()

4      178628
5      170230
3      166368
6      165882
7      152537
        ...  
124       124
121       121
111       111
107       107
97         97
Name: customer_frequency, Length: 163, dtype: int64

In [123]:
customer_frequencies['customer_frequency'] = (customer_frequencies['customer_frequency']-customer_frequencies['customer_frequency'].min())/(customer_frequencies['customer_frequency'].max()-customer_frequencies['customer_frequency'].min())

In [124]:
customer_frequencies['customer_frequency'].value_counts()

0.010417    178628
0.013889    170230
0.006944    166368
0.017361    165882
0.020833    152537
             ...  
0.427083       124
0.416667       121
0.381944       111
0.368056       107
0.333333        97
Name: customer_frequency, Length: 163, dtype: int64

In [125]:
customer_frequencies.shape[0]

2762872

In [126]:
customer_frequencies.drop_duplicates(inplace=True)

In [127]:
customer_frequencies.shape[0]

437365

#### Customer Average Article Cost

In [128]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased
29030503,2020-07-15,272412481300040,778064028,0.008458,1,95,1
29030504,2020-07-15,272412481300040,816592008,0.016932,1,95,1
29030505,2020-07-15,272412481300040,621381021,0.033881,1,95,1
29030506,2020-07-15,272412481300040,817477003,0.025407,1,95,1
29030507,2020-07-15,272412481300040,899088002,0.025407,1,95,1


In [129]:
average_cost = transactions[['customer_id', 'price']].copy()
average_cost['average_article_cost'] = average_cost.groupby(['customer_id']).transform('mean')
average_cost = average_cost[['customer_id', 'average_article_cost']]
average_cost.drop_duplicates(inplace=True)
average_cost.head()

,customer_id,average_article_cost
29030503,272412481300040,0.030597
29064059,1456826891333599,0.016206
29067103,2133687643102426,0.042356
29027487,6010692573790711,0.026367
29046403,6171059100114610,0.018062


#### Detailed description vector

In [130]:
#!pip install -U sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [132]:
from sentence_transformers import SentenceTransformer, util

In [133]:
articles_desc = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv', usecols=['article_id', 'detail_desc'])

In [134]:
articles_desc

,article_id,detail_desc
0,108775015,Jersey top with narrow shoulder straps.
1,108775044,Jersey top with narrow shoulder straps.
2,108775051,Jersey top with narrow shoulder straps.
3,110065001,"Microfibre T-shirt bra with underwired, moulde..."
4,110065002,"Microfibre T-shirt bra with underwired, moulde..."
...,...,...
105537,953450001,Socks in a fine-knit cotton blend with a small...
105538,953763001,Loose-fitting sports vest top in ribbed fast-d...
105539,956217002,"Short, A-line dress in jersey with a round nec..."
105540,957375001,Large plastic hair claw.


In [135]:
articles_desc.dropna(axis=0, inplace=True)
articles_desc

,article_id,detail_desc
0,108775015,Jersey top with narrow shoulder straps.
1,108775044,Jersey top with narrow shoulder straps.
2,108775051,Jersey top with narrow shoulder straps.
3,110065001,"Microfibre T-shirt bra with underwired, moulde..."
4,110065002,"Microfibre T-shirt bra with underwired, moulde..."
...,...,...
105537,953450001,Socks in a fine-knit cotton blend with a small...
105538,953763001,Loose-fitting sports vest top in ribbed fast-d...
105539,956217002,"Short, A-line dress in jersey with a round nec..."
105540,957375001,Large plastic hair claw.


In [136]:
description_model = SentenceTransformer('all-MiniLM-L6-v2')

In [137]:
#converting to list of descriptions to be passed to encoder
descriptions = articles_desc['detail_desc'].tolist()

In [138]:
#encoding
embeddings = description_model.encode(descriptions)

Batches:   0%|          | 0/3286 [00:00<?, ?it/s]

In [139]:
embeddings.shape

(105126, 384)

In [140]:
#PCA so we arent adding 384 new features - top 2 principle components
from sklearn.decomposition import PCA

In [141]:
description_PCA = PCA(n_components=2)
principle_components = description_PCA.fit_transform(embeddings)

In [142]:
principle_components.shape

(105126, 2)

In [143]:
articles_desc["desc_pc1"],articles_desc["desc_pc2"] = principle_components[:,0], principle_components[:,1]

In [144]:
articles_desc.head()

,article_id,detail_desc,desc_pc1,desc_pc2
0,108775015,Jersey top with narrow shoulder straps.,-0.248286,0.305955
1,108775044,Jersey top with narrow shoulder straps.,-0.248288,0.305953
2,108775051,Jersey top with narrow shoulder straps.,-0.248288,0.305954
3,110065001,"Microfibre T-shirt bra with underwired, moulde...",-0.150632,0.103688
4,110065002,"Microfibre T-shirt bra with underwired, moulde...",-0.150632,0.103687


In [145]:
articles_desc = articles_desc[['article_id', 'desc_pc1', 'desc_pc2']]

In [146]:
articles_desc.head()

,article_id,desc_pc1,desc_pc2
0,108775015,-0.248286,0.305955
1,108775044,-0.248288,0.305953
2,108775051,-0.248288,0.305954
3,110065001,-0.150632,0.103688
4,110065002,-0.150632,0.103687


### Add bestseller information

In [147]:
data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
)

In [148]:
data = data[data.week != data.week.min()]
data.bestseller_rank.fillna(999, inplace=True)

In [149]:
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')

In [150]:
#adding my feature: customer frequency
data = pd.merge(data, customer_frequencies, on='customer_id', how='left')

In [151]:
#adding my feature: per-customer average article cost
data = pd.merge(data, average_cost, on='customer_id', how='left')

In [152]:
#adding my feature: description embeddings principle components
data = pd.merge(data, articles_desc, on='article_id', how='left')

In [153]:
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [154]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [155]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [156]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'bestseller_rank', 'desc_pc1', 'desc_pc2', 'average_article_cost', 'customer_frequency']

In [157]:
%%time

train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

CPU times: user 364 ms, sys: 260 ms, total: 624 ms
Wall time: 622 ms


# Model training

In [158]:
from lightgbm.sklearn import LGBMRanker

In [159]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=1,
    importance_type='gain',
    verbose=10
)

In [160]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.848850
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.128405
[LightGBM] [Debug] init for col-wise cost 0.258794 seconds, init for row-wise cost 1.557768 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.744455 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 2002
[LightGBM] [Info] Number of data points in the train set: 11381612, number of used features: 22
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
CPU times: user 19.2 s, sys: 2.21 s, total: 21.5 s
Wall time: 12.6 s


In [161]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

bestseller_rank 0.9899945041136253
customer_frequency 0.009377166520435314
article_id 0.0002940579744400044
section_no 9.394018951457532e-05
average_article_cost 7.624874864650692e-05
garment_group_no 4.9681431971225006e-05
desc_pc1 3.379320497509625e-05
age 3.1820934364166175e-05
department_no 3.1464232106512994e-05
desc_pc2 1.7322649921282507e-05
fashion_news_frequency 0.0
club_member_status 0.0
Active 0.0
FN 0.0
postal_code 0.0
index_group_no 0.0
index_code 0.0
perceived_colour_master_id 0.0
perceived_colour_value_id 0.0
colour_group_code 0.0
graphical_appearance_no 0.0
product_type_no 0.0


# Calculate predictions

In [162]:
%time

test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs


# Create submission

In [163]:
sub = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [164]:
%%time
preds = []
for c_id in customer_hex_id_to_int(sub.customer_id):
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    preds.append(pred[:12])

CPU times: user 6.35 s, sys: 749 ms, total: 7.1 s
Wall time: 7.09 s


In [165]:
preds = [' '.join(['0' + str(p) for p in ps]) for ps in preds]
sub.prediction = preds

In [166]:
sub_name = 'basic_model_submission-with_all'
sub.to_csv(f'{sub_name}.csv.gz', index=False)